<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Visualizaci%C3%B3n_Integrada_Multiples_Instrumentos_Version_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MONITOREO INSTRUMENTACIÓN PRESAS

#**Carga Masiva de datos Crudos**

In [5]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

#Visualización Integrada de Multiples Instrumentos

##**Instalación de dependencias para generar las visualizaciones**

In [4]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

##**Generador de gráficas de multiples instrumentos**

# Visualización Interactiva de Instrumentos de Auscultación

Este panel interactivo permite visualizar datos de **Puntos Fijos** y **Piezómetros Eléctricos** en gráficos personalizados.

## ✅ ¿Qué puedes hacer?

- Seleccionar el tipo de instrumento (Puntos Fijos o Piezómetros Eléctricos)
- Elegir el origen de los datos: archivos `.CSV` o `.XLSX`
- Filtrar por:
  - Margen (MI / MD) o Progresiva
  - Instrumento específico (Punto Fijo o Piezómetro)
  - Variable de medición
  - Año
- Elegir el estilo del gráfico, grosor de línea, tamaño y paleta de colores
- Generar gráficos interactivos con `Plotly`
- Descargar los gráficos en formatos: PNG, JPEG, PDF, SVG o HTML

## 📊 Cómo usarlo

1. Sube tus archivos desde el cargador (si es necesario).
2. Selecciona el **instrumento** a visualizar.
3. Filtra las opciones disponibles.
4. Haz clic en **"Graficar"**.
5. Si deseas guardar la imagen, selecciona el formato y pulsa **"Guardar gráfica"**.

## 🔁 ¿Qué pasa si cambio de instrumento?

- Los selectores cambian dinámicamente dependiendo de si seleccionas:
  - Puntos Fijos → muestra `Margen` y `Punto Fijo`
  - Piezómetros → muestra `Progresiva` y `Piezómetro`

## 📌 Notas

- Si ves múltiples gráficos uno debajo del otro, asegúrate de que se esté usando `clear_output()` en el código para limpiar salidas previas.
- Asegúrate de tener los datos correctamente formateados y cargados en los diccionarios `datos_csv` o `datos_xlsx`.

---


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores Comunes de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por tipo de instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores según Instrumento ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for widget in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
                   progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown]:
        widget.layout.display = 'none'

    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    else:
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()

# === Datos cargados externos (asegúrate de tener estos dicts ya cargados) ===
# datos_csv = {"puntos_fijos_mi": ..., "puntos_fijos_md": ..., "piezometros_electricos": ...}
# datos_xlsx = {igual}

# === Actualizar selectores PF ===
def actualizar_opciones_pf():
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {
        "Margen Izquierda (MI)": df_mi.copy(),
        "Margen Derecha (MD)": df_md.copy()
    }
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        return

    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

    columnas_excluir = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluir]
    variable_pf_dropdown.options = variables
    if variables:
        variable_pf_dropdown.value = variables[0]

    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Actualizar selectores PE ===
def actualizar_opciones_pe():
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty:
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown.value = progresiva_dropdown.options[0]

    actualizar_piezometros_pe()
    columnas_excluir = ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']
    variables = [c for c in df.select_dtypes(include='number').columns if c not in columnas_excluir]
    variable_pe_dropdown.options = variables
    if variables:
        variable_pe_dropdown.value = variables[0]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pe_dropdown.options = ["Todos"] + [str(a) for a in anios]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    prog = progresiva_dropdown.value
    piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + piezos

# === Gráfica ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)

    if instrumento_dropdown.value == "Puntos Fijos":
        origen = origen_dropdown.value
        df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
        df = df.copy()
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable_pf_dropdown.value])

        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value]
        titulo = f"{margen_dropdown.value}: {variable_pf_dropdown.value}"

        instrumento_col = 'INSTRUMENTO'
        variable = variable_pf_dropdown.value

    else:
        df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
        df = df.copy()
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
        if piezometro_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
        if anio_pe_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
        df = df.dropna(subset=['FECHA', variable_pe_dropdown.value])
        titulo = f"{progresiva_dropdown.value} – {variable_pe_dropdown.value}"

        instrumento_col = 'PIEZOMETRO'
        variable = variable_pe_dropdown.value

    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[instrumento_col].unique())
    colores = {pf: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, pf in enumerate(elementos)}
    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        d = df[df[instrumento_col] == elem]
        fig.add_trace(go.Scatter(
            x=d['FECHA'], y=d[variable],
            mode=modo_graf,
            name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem]),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup,
            line_shape=line_shape
        ))

    ancho, alto = tamanio_dropdown.value
    fig.update_layout(
        width=ancho, height=alto,
        title=titulo,
        xaxis_title="Fecha",
        yaxis_title=variable,
        hovermode="x unified",
        legend_title=instrumento_col
    )
    fig.show()

# === Guardar gráfica ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero debes generar la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala 'kaleido':\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error:", e)

# === Observadores ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === Mostrar interfaz ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([margen_dropdown, punto_dropdown, progresiva_dropdown, piezometro_dropdown]))
display(widgets.HBox([variable_pf_dropdown, variable_pe_dropdown, anio_pf_dropdown, anio_pe_dropdown]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar visibilidad
actualizar_controles_visibles()


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()

# Visualización Interactiva de Instrumentos Geotécnicos

Este panel permite analizar gráficamente los datos de:

- **Puntos Fijos**
- **Piezómetros Eléctricos**
- **Piezómetros Casagrande**

### Características:
- Filtros por **origen** (CSV/XLSX), **instrumento**, **ubicación**, **año**, **variable**.
- Estilos de gráfico: spline, líneas, puntos, áreas.
- Configuración de tamaño, grosor y paleta de colores.
- **Botón para guardar** la figura como imagen (.png, .jpg, .svg, .pdf) o archivo interactivo (.html).

> Los datos deben estar previamente cargados en los diccionarios `datos_csv` y `datos_xlsx` según formato y tipo.


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por Instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Casagrande
margen_cg_dropdown = widgets.Dropdown(description="Margen:")
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_cg_dropdown = widgets.Dropdown(description="Variable:")
anio_cg_dropdown = widgets.Dropdown(description="Año:")

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown]:
        w.layout.display = 'none'

    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()

# === Datos cargados externamente ===
# datos_csv = {"puntos_fijos_mi": ..., "piezometros_casagrande": ..., etc.}
# datos_xlsx = {...}

# === Puntos Fijos ===
def actualizar_opciones_pf():
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi.copy(), "Margen Derecha (MD)": df_md.copy()}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets: return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Piezómetros Eléctricos ===
def actualizar_opciones_pe():
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    piezometro_dropdown.options = ["Todos"] + sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique())

# === Piezómetros Casagrande ===
def actualizar_opciones_cg():
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique())
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    df = df[df['MARGEN'] == margen_cg_dropdown.value]
    pz_cg_dropdown.options = ["Todos"] + sorted(df['PIEZOMETRO'].dropna().unique())

margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')

# === GRAFICAR ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
    tipo = instrumento_dropdown.value
    origen = origen_dropdown.value

    if tipo == "Puntos Fijos":
        df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
        variable, col = variable_pf_dropdown.value, 'INSTRUMENTO'
        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value]
        titulo = f"{margen_dropdown.value}: {variable}"

    elif tipo == "Piezómetros Eléctricos":
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
        variable, col = variable_pe_dropdown.value, 'PIEZOMETRO'
        if piezometro_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
        if anio_pe_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
        titulo = f"{progresiva_dropdown.value} – {variable}"

    elif tipo == "Piezómetros Casagrande":
        df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
        df = df[df['MARGEN'] == margen_cg_dropdown.value]
        variable, col = variable_cg_dropdown.value, 'PIEZOMETRO'
        if pz_cg_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
        if anio_cg_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
        titulo = f"{margen_cg_dropdown.value} – {variable}"

    else:
        print("Instrumento no reconocido")
        return

    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['FECHA', variable])
    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[col].unique())
    colores = {e: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, e in enumerate(elementos)}
    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        subdf = df[df[col] == elem]
        fig.add_trace(go.Scatter(
            x=subdf['FECHA'], y=subdf[variable],
            mode=modo_graf, name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem]),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup,
            line_shape=line_shape
        ))

    fig.update_layout(
        width=tamanio_dropdown.value[0], height=tamanio_dropdown.value[1],
        title=titulo, xaxis_title="Fecha", yaxis_title=variable,
        hovermode="x unified", legend_title=col
    )
    fig.show()

# === GUARDAR ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext): ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero generá la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instalá kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === DISPLAY FINAL ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown
]))
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))
display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar
actualizar_controles_visibles()


Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()

# Visualización Interactiva de Instrumentos Geotécnicos

Este panel permite analizar gráficamente los datos de:

- **Puntos Fijos**
- **Piezómetros Eléctricos**
- **Piezómetros Casagrande**
- **Inclinometros**

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por Instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Casagrande
margen_cg_dropdown = widgets.Dropdown(description="Margen:")
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_cg_dropdown = widgets.Dropdown(description="Variable:")
anio_cg_dropdown = widgets.Dropdown(description="Año:")

# Inclinómetros
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:")
anio_inc_dropdown = widgets.Dropdown(description="Año:")
eje_dropdown = widgets.Dropdown(
    options=["A+", "A-", "B+", "B-"],
    value="A+", description="Eje:"
)

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value

    # Ocultar todos los selectores específicos
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown]:
        w.layout.display = 'none'

    # Mostrar selectores según el instrumento seleccionado
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()

# === Datos cargados externamente ===
# datos_csv = {"puntos_fijos_mi": ..., "piezometros_casagrande": ..., "inclinometros": ..., etc.}
# datos_xlsx = {...}

# === Puntos Fijos ===
def actualizar_opciones_pf():
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi.copy(), "Margen Derecha (MD)": df_md.copy()}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets: return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Piezómetros Eléctricos ===
def actualizar_opciones_pe():
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    piezometro_dropdown.options = ["Todos"] + sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique())

# === Piezómetros Casagrande ===
def actualizar_opciones_cg():
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique())
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    df = df[df['MARGEN'] == margen_cg_dropdown.value]
    pz_cg_dropdown.options = ["Todos"] + sorted(df['PIEZOMETRO'].dropna().unique())

# === Inclinómetros ===
def actualizar_opciones_inc():
    origen = origen_dropdown.value
    df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
    if df.empty:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = []
        return
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = sorted(df['Inclinometro'].dropna().unique())
    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_inc_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Eventos para actualizar opciones dependientes ===
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')

# === GRAFICAR ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)

    tipo = instrumento_dropdown.value
    origen = origen_dropdown.value

    if tipo == "Puntos Fijos":
        df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
        variable, col = variable_pf_dropdown.value, 'INSTRUMENTO'
        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value]
        titulo = f"{margen_dropdown.value}: {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Piezómetros Eléctricos":
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
        df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
        variable, col = variable_pe_dropdown.value, 'PIEZOMETRO'
        if piezometro_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
        if anio_pe_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
        titulo = f"{progresiva_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Piezómetros Casagrande":
        df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
        df = df[df['MARGEN'] == margen_cg_dropdown.value]
        variable, col = variable_cg_dropdown.value, 'PIEZOMETRO'
        if pz_cg_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
        if anio_cg_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
        titulo = f"{margen_cg_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Inclinómetros":
        df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
        df = df.copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
        eje = eje_dropdown.value
        inc = inclinometro_dropdown.value
        anio = anio_inc_dropdown.value

        df = df[df['Inclinometro'] == inc]
        if anio != "Todos":
            df = df[df['Fecha'].dt.year == int(anio)]

        if df.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        # Para inclinómetros, graficamos de manera especial (perfil de profundidad)
        fig = go.Figure()
        fechas = sorted(df['Fecha'].dropna().unique())
        paleta = paleta_dropdown.value

        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for i, fecha in enumerate(fechas):
            d = df[df['Fecha'] == fecha].copy()
            if eje not in d.columns or 'Profundidad' not in d.columns:
                continue
            d = d.dropna(subset=[eje, 'Profundidad'])
            if d.empty:
                continue
            fig.add_trace(go.Scatter(
                x=d[eje], y=d['Profundidad'],
                mode=modo_graf,
                name=str(fecha.date()),
                line=dict(width=grosor_dropdown.value, color=paleta[i % len(paleta)], shape=line_shape),
                marker=dict(color=paleta[i % len(paleta)]),
                fill=fill, stackgroup=stackgroup
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{inc} – Perfil {eje}",
            xaxis_title=f"Deformación ({eje})",
            yaxis_title="Profundidad (m)",
            yaxis_autorange="reversed",
            hovermode="closest",
            legend_title="Fecha"
        )
        fig.show()
        return

    else:
        print("Instrumento no reconocido")
        return

    # Para instrumentos que no sean inclinómetros
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['FECHA', variable])
    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[col].unique())
    colores = {e: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, e in enumerate(elementos)}

    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        subdf = df[df[col] == elem]
        fig.add_trace(go.Scatter(
            x=subdf[x_col], y=subdf[y_col],
            mode=modo_graf, name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem], shape=line_shape),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup
        ))

    fig.update_layout(
        width=tamanio_dropdown.value[0], height=tamanio_dropdown.value[1],
        title=titulo, xaxis_title=x_title, yaxis_title=y_title,
        hovermode="x unified", legend_title=col
    )
    fig.show()

# === GUARDAR ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero generá la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instalá kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === DISPLAY FINAL ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))

# Fila 1: Selectores principales por instrumento
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown
]))

# Fila 2: Variables y años
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown
]))

# Fila 3: Opciones de gráfica
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))

display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar
actualizar_controles_visibles()

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()

# Visualización Interactiva de Instrumentos Geotécnicos

Este script permite visualizar datos de auscultación para distintos instrumentos geotécnicos:

- **Puntos Fijos**
- **Piezómetros Eléctricos y Casagrande**
- **Inclinómetros**
- **Celdas de Asentamiento**
- **Freatímetros**
- **Extensómetros**

## Funcionalidades incluidas

- Selectores dinámicos de instrumento, variable, punto/progresiva, año y estilo gráfico.
- Gráficas interactivas con `Plotly`.
- Selección de paleta de colores y tamaño.
- Opción para **guardar la gráfica** en distintos formatos (`.png`, `.jpg`, `.svg`, `.pdf`, `.html`).
- Se puede elegir dónde guardar con una **ventana emergente del sistema**.

> **Nota:** Para guardar imágenes se requiere instalar `kaleido` si no está disponible:
```bash
%pip install -U kaleido


In [9]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por Instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Casagrande
margen_cg_dropdown = widgets.Dropdown(description="Margen:")
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_cg_dropdown = widgets.Dropdown(description="Variable:")
anio_cg_dropdown = widgets.Dropdown(description="Año:")

# Inclinómetros
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:")
anio_inc_dropdown = widgets.Dropdown(description="Año:")
eje_dropdown = widgets.Dropdown(
    options=["A+", "A-", "B+", "B-"],
    value="A+", description="Eje:"
)

# Celdas de Asentamiento
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:")
celda_dropdown = widgets.Dropdown(description="Celda:")
variable_ca_dropdown = widgets.Dropdown(description="Variable:")
anio_ca_dropdown = widgets.Dropdown(description="Año:")

# Freatímetros
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:")
variable_fr_dropdown = widgets.Dropdown(description="Variable:")
anio_fr_dropdown = widgets.Dropdown(description="Año:")

# Extensómetros
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:")
variable_ex_dropdown = widgets.Dropdown(description="Variable:")
anio_ex_dropdown = widgets.Dropdown(description="Año:")

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value

    # Ocultar todos los selectores específicos
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'

    # Mostrar selectores según el instrumento seleccionado
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# === Puntos Fijos ===
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
    df_md = (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = []
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Piezómetros Eléctricos ===
def actualizar_opciones_pe():
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    piezometro_dropdown.options = ["Todos"] + sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique())

# === Piezómetros Casagrande ===
def actualizar_opciones_cg():
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique())
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    df = df[df['MARGEN'] == margen_cg_dropdown.value].copy()
    pz_cg_dropdown.options = ["Todos"] + sorted(df['PIEZOMETRO'].dropna().unique())

# === Inclinómetros ===
def actualizar_opciones_inc():
    origen = origen_dropdown.value
    df = (datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]).copy()
    if df.empty:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = []
        return
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = sorted(df['Inclinometro'].dropna().unique())
    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_inc_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Celdas de Asentamiento ===
def actualizar_opciones_ca():
    df = (datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]).copy()
    if df.empty:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = (datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    prog = progresiva_ca_dropdown.value
    celdas = sorted(df[df['PROGRESIVA'] == prog]['CELDA_DE_ASENTAMIENTO'].dropna().unique())
    celda_dropdown.options = ["Todas"] + list(celdas)

# === Freatímetros ===
def actualizar_opciones_fr():
    df = (datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]).copy()
    if df.empty:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Extensómetros ===
def actualizar_opciones_ex():
    df = (datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]).copy()
    if df.empty:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Eventos para actualizar opciones dependientes ===
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')

# === GRAFICAR ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)

    tipo = instrumento_dropdown.value
    origen = origen_dropdown.value

    if tipo == "Puntos Fijos":
        df = (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
        variable, col = variable_pf_dropdown.value, 'INSTRUMENTO'
        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)].copy()
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value].copy()
        titulo = f"{margen_dropdown.value}: {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Piezómetros Eléctricos":
        df = (datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
        df = df[df['PROGRESIVA'] == progresiva_dropdown.value].copy()
        variable, col = variable_pe_dropdown.value, 'PIEZOMETRO'
        if piezometro_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown.value].copy()
        if anio_pe_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)].copy()
        titulo = f"{progresiva_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Piezómetros Casagrande":
        df = (datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
        df = df[df['MARGEN'] == margen_cg_dropdown.value].copy()
        variable, col = variable_cg_dropdown.value, 'PIEZOMETRO'
        if pz_cg_dropdown.value != "Todos":
            df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value].copy()
        if anio_cg_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)].copy()
        titulo = f"{margen_cg_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Inclinómetros":
        df = (datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
        eje = eje_dropdown.value
        inc = inclinometro_dropdown.value
        anio = anio_inc_dropdown.value

        df = df[df['Inclinometro'] == inc].copy()
        if anio != "Todos":
            df = df[df['Fecha'].dt.year == int(anio)].copy()

        if df.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        # Para inclinómetros, graficamos de manera especial (perfil de profundidad)
        fig = go.Figure()
        fechas = sorted(df['Fecha'].dropna().unique())
        paleta = paleta_dropdown.value

        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for i, fecha in enumerate(fechas):
            d = df[df['Fecha'] == fecha].copy()
            if eje not in d.columns or 'Profundidad' not in d.columns:
                continue
            d = d.dropna(subset=[eje, 'Profundidad'])
            if d.empty:
                continue
            fig.add_trace(go.Scatter(
                x=d[eje], y=d['Profundidad'],
                mode=modo_graf,
                name=str(fecha.date()),
                line=dict(width=grosor_dropdown.value, color=paleta[i % len(paleta)], shape=line_shape),
                marker=dict(color=paleta[i % len(paleta)]),
                fill=fill, stackgroup=stackgroup
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{inc} – Perfil {eje}",
            xaxis_title=f"Deformación ({eje})",
            yaxis_title="Profundidad (m)",
            yaxis_autorange="reversed",
            hovermode="closest",
            legend_title="Fecha"
        )
        fig.show()
        return

    elif tipo == "Celdas de Asentamiento":
        df = (datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]).copy()
        df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value].copy()
        variable, col = variable_ca_dropdown.value, 'CELDA_DE_ASENTAMIENTO'
        if celda_dropdown.value != "Todas":
            df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value].copy()
        if anio_ca_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)].copy()
        titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Freatímetros":
        df = (datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]).copy()
        variable, col = variable_fr_dropdown.value, 'FREATIMETRO'
        if freatimetro_dropdown.value != "Todos":
            df = df[df['FREATIMETRO'] == freatimetro_dropdown.value].copy()
        if anio_fr_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)].copy()
        titulo = f"{variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Extensómetros":
        df = (datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]).copy()
        variable, col = variable_ex_dropdown.value, 'EXTENSOMETRO'
        if extensometro_dropdown.value != "Todos":
            df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value].copy()
        if anio_ex_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)].copy()
        titulo = f"{variable} por Extensómetro"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    else:
        print("Instrumento no reconocido")
        return

    # Para instrumentos que no sean inclinómetros
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['FECHA', variable]).copy()
    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[col].unique())
    colores = {e: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, e in enumerate(elementos)}

    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        subdf = df[df[col] == elem].copy()
        fig.add_trace(go.Scatter(
            x=subdf[x_col], y=subdf[y_col],
            mode=modo_graf, name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem], shape=line_shape),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup
        ))

    fig.update_layout(
        width=tamanio_dropdown.value[0], height=tamanio_dropdown.value[1],
        title=titulo, xaxis_title=x_title, yaxis_title=y_title,
        hovermode="x unified", legend_title=col
    )
    fig.show()

# === GUARDAR ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero generá la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instalá kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === DISPLAY FINAL ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))

# Fila 1: Selectores principales por instrumento
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))

# Fila 2: Variables y años
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))

# Fila 3: Opciones de gráfica
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))

display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar
actualizar_controles_visibles()

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()

#Se actualiza el Script para incluir las variables COTAS_NF para Visualizar en el caso de los Piezómetros Eléctricos y Piezómetros Casagrande

In [11]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# === Selectores Comunes ===
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

# === Selectores de Gráfica ===
estilo_dropdown = widgets.Dropdown(
    options=[
        "Curvas suaves (spline)", "Líneas rectas", "Puntos",
        "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
    ],
    value="Curvas suaves (spline)", description="Estilo:"
)
grosor_dropdown = widgets.Dropdown(
    options={"Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4, "Grueso (7px)": 7, "Extra grueso (10px)": 10},
    value=2, description="Grosor:"
)
tamanio_dropdown = widgets.Dropdown(
    options={"Pequeño": (600, 400), "Mediano": (900, 500), "Grande": (1200, 700), "Extra grande": (1600, 1000)},
    value=(900, 500), description="Tamaño:"
)
paleta_dropdown = widgets.Dropdown(
    options={
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    },
    value=pc.qualitative.Plotly, description="Paleta:"
)

# === Selectores por Instrumento ===
# Puntos Fijos
margen_dropdown = widgets.Dropdown(description="Margen:")
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_pf_dropdown = widgets.Dropdown(description="Variable:")
anio_pf_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Eléctricos
progresiva_dropdown = widgets.Dropdown(description="Progresiva:")
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_pe_dropdown = widgets.Dropdown(description="Variable:")
anio_pe_dropdown = widgets.Dropdown(description="Año:")

# Piezómetros Casagrande
margen_cg_dropdown = widgets.Dropdown(description="Margen:")
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:")
variable_cg_dropdown = widgets.Dropdown(description="Variable:")
anio_cg_dropdown = widgets.Dropdown(description="Año:")

# Inclinómetros
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:")
anio_inc_dropdown = widgets.Dropdown(description="Año:")
eje_dropdown = widgets.Dropdown(
    options=["A+", "A-", "B+", "B-"],
    value="A+", description="Eje:"
)

# Celdas de Asentamiento
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:")
celda_dropdown = widgets.Dropdown(description="Celda:")
variable_ca_dropdown = widgets.Dropdown(description="Variable:")
anio_ca_dropdown = widgets.Dropdown(description="Año:")

# Freatímetros
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:")
variable_fr_dropdown = widgets.Dropdown(description="Variable:")
anio_fr_dropdown = widgets.Dropdown(description="Año:")

# Extensómetros
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:")
variable_ex_dropdown = widgets.Dropdown(description="Variable:")
anio_ex_dropdown = widgets.Dropdown(description="Año:")

# === Botones y salida ===
boton = widgets.Button(description="Graficar", button_style="success")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
ruta_text = widgets.Text(value="grafica", description="Ruta y nombre:")
formato_dropdown = widgets.Dropdown(
    options={"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"},
    value=".png", description="Formato:"
)
output = widgets.Output()
output_guardar = widgets.Output()

# === Mostrar/Ocultar Selectores ===
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value

    # Ocultar todos los selectores específicos
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'

    # Mostrar selectores según el instrumento seleccionado
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# === Puntos Fijos ===
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
    df_md = (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = []
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Piezómetros Eléctricos ===
def actualizar_opciones_pe(change=None):
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    columnas_excluir = ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']
    variables = [c for c in df.columns if c not in columnas_excluir]
    variable_pe_dropdown.options = variables
    if variables:
        variable_pe_dropdown.value = variables[0]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = (datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown.options = ["Todos"] + list(piezos)
    if piezometro_dropdown.options:
        piezometro_dropdown.value = piezometro_dropdown.options[0]

# === Piezómetros Casagrande ===
def actualizar_opciones_cg(change=None):
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    if df.empty or 'MARGEN' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    columnas_excluir = ['FECHA', 'MARGEN', 'PIEZOMETRO']
    variables = [c for c in df.columns if c not in columnas_excluir]
    variable_cg_dropdown.options = variables
    if variables:
        variable_cg_dropdown.value = variables[0]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = (datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)
    if pz_cg_dropdown.options:
        pz_cg_dropdown.value = pz_cg_dropdown.options[0]

# === Inclinómetros ===
def actualizar_opciones_inc():
    origen = origen_dropdown.value
    df = (datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]).copy()
    if df.empty:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = []
        return
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = sorted(df['Inclinometro'].dropna().unique())
    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_inc_dropdown.options = ["Todos"] + [str(a) for a in anios]

# === Celdas de Asentamiento ===
def actualizar_opciones_ca():
    df = (datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]).copy()
    if df.empty:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique())
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = (datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]).copy()
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    prog = progresiva_ca_dropdown.value
    celdas = sorted(df[df['PROGRESIVA'] == prog]['CELDA_DE_ASENTAMIENTO'].dropna().unique())
    celda_dropdown.options = ["Todas"] + list(celdas)

# === Freatímetros ===
def actualizar_opciones_fr():
    df = (datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]).copy()
    if df.empty:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Extensómetros ===
def actualizar_opciones_ex():
    df = (datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]).copy()
    if df.empty:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# === Eventos para actualizar opciones dependientes ===
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')

# === GRAFICAR ===
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)

    tipo = instrumento_dropdown.value
    origen = origen_dropdown.value

    if tipo == "Puntos Fijos":
        df = (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
        if margen_dropdown.value == "Margen Derecha (MD)":
            df = (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
        variable, col = variable_pf_dropdown.value, 'INSTRUMENTO'
        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)].copy()
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value].copy()
        titulo = f"{margen_dropdown.value}: {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Piezómetros Eléctricos":
        df = (datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        prog = progresiva_dropdown.value
        piezo = piezometro_dropdown.value
        variable = variable_pe_dropdown.value
        anio = anio_pe_dropdown.value

        df_plot = df[df['PROGRESIVA'] == prog].copy()
        if piezo != "Todos":
            df_plot = df_plot[df_plot['PIEZOMETRO'] == piezo].copy()
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)].copy()

        # Conversión segura para COTA_NF
        if variable == "COTA_NF":
            df_plot[variable] = pd.to_numeric(df_plot[variable], errors="coerce")

        df_plot = df_plot.dropna(subset=['FECHA', variable, 'PIEZOMETRO'])
        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['PIEZOMETRO'].unique())
        color_map = {pz: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, pz in enumerate(instrumentos)}

        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for pz in instrumentos:
            datos_pz = df_plot[df_plot['PIEZOMETRO'] == pz]
            fig.add_trace(go.Scatter(
                x=datos_pz['FECHA'],
                y=datos_pz[variable],
                mode=modo_graf,
                name=pz,
                line=dict(width=grosor_dropdown.value, color=color_map[pz], shape=line_shape),
                marker=dict(color=color_map[pz]),
                fill=fill,
                stackgroup=stackgroup
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{prog} – {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Piezómetro",
            hovermode="x unified"
        )
        fig.show()
        return

    elif tipo == "Piezómetros Casagrande":
        df = (datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        margen = margen_cg_dropdown.value
        piezo = pz_cg_dropdown.value
        variable = variable_cg_dropdown.value
        anio = anio_cg_dropdown.value

        df_plot = df[df['MARGEN'] == margen].copy()
        if piezo != "Todos":
            df_plot = df_plot[df_plot['PIEZOMETRO'] == piezo].copy()
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)].copy()

        # Conversión segura para COTA_NF
        if variable == "COTA_NF":
            df_plot[variable] = pd.to_numeric(df_plot[variable], errors="coerce")

        df_plot = df_plot.dropna(subset=['FECHA', variable, 'PIEZOMETRO'])
        if df_plot.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        fig = go.Figure()
        instrumentos = sorted(df_plot['PIEZOMETRO'].unique())
        color_map = {pz: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, pz in enumerate(instrumentos)}

        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for pz in instrumentos:
            datos_pz = df_plot[df_plot['PIEZOMETRO'] == pz]
            fig.add_trace(go.Scatter(
                x=datos_pz['FECHA'],
                y=datos_pz[variable],
                mode=modo_graf,
                name=pz,
                line=dict(width=grosor_dropdown.value, color=color_map[pz], shape=line_shape),
                marker=dict(color=color_map[pz]),
                fill=fill,
                stackgroup=stackgroup
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{margen} – {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Piezómetro",
            hovermode="x unified"
        )
        fig.show()
        return

    elif tipo == "Inclinómetros":
        df = (datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]).copy()
        if df.empty:
            print("⚠️ No hay datos disponibles.")
            return

        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
        eje = eje_dropdown.value
        inc = inclinometro_dropdown.value
        anio = anio_inc_dropdown.value

        df = df[df['Inclinometro'] == inc].copy()
        if anio != "Todos":
            df = df[df['Fecha'].dt.year == int(anio)].copy()

        if df.empty:
            print("⚠️ No hay datos para graficar con esa selección.")
            return

        # Para inclinómetros, graficamos de manera especial (perfil de profundidad)
        fig = go.Figure()
        fechas = sorted(df['Fecha'].dropna().unique())
        paleta = paleta_dropdown.value

        modo = {
            "Curvas suaves (spline)": ("lines", "spline"),
            "Líneas rectas": ("lines", "linear"),
            "Puntos": ("markers", None),
            "Líneas + Puntos": ("lines+markers", "linear"),
            "Área apilada": ("lines", "linear"),
            "Área + Líneas": ("lines", "linear"),
            "Área + Líneas + Puntos": ("lines+markers", "linear")
        }
        modo_graf, line_shape = modo[estilo_dropdown.value]
        fill = "tozeroy" if "Área" in estilo_dropdown.value else None
        stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

        for i, fecha in enumerate(fechas):
            d = df[df['Fecha'] == fecha].copy()
            if eje not in d.columns or 'Profundidad' not in d.columns:
                continue
            d = d.dropna(subset=[eje, 'Profundidad'])
            if d.empty:
                continue
            fig.add_trace(go.Scatter(
                x=d[eje], y=d['Profundidad'],
                mode=modo_graf,
                name=str(fecha.date()),
                line=dict(width=grosor_dropdown.value, color=paleta[i % len(paleta)], shape=line_shape),
                marker=dict(color=paleta[i % len(paleta)]),
                fill=fill, stackgroup=stackgroup
            ))

        ancho, alto = tamanio_dropdown.value
        fig.update_layout(
            width=ancho, height=alto,
            title=f"{inc} – Perfil {eje}",
            xaxis_title=f"Deformación ({eje})",
            yaxis_title="Profundidad (m)",
            yaxis_autorange="reversed",
            hovermode="closest",
            legend_title="Fecha"
        )
        fig.show()
        return

    elif tipo == "Celdas de Asentamiento":
        df = (datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]).copy()
        df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value].copy()
        variable, col = variable_ca_dropdown.value, 'CELDA_DE_ASENTAMIENTO'
        if celda_dropdown.value != "Todas":
            df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value].copy()
        if anio_ca_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)].copy()
        titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Freatímetros":
        df = (datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]).copy()
        variable, col = variable_fr_dropdown.value, 'FREATIMETRO'
        if freatimetro_dropdown.value != "Todos":
            df = df[df['FREATIMETRO'] == freatimetro_dropdown.value].copy()
        if anio_fr_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)].copy()
        titulo = f"{variable}"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    elif tipo == "Extensómetros":
        df = (datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]).copy()
        variable, col = variable_ex_dropdown.value, 'EXTENSOMETRO'
        if extensometro_dropdown.value != "Todos":
            df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value].copy()
        if anio_ex_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)].copy()
        titulo = f"{variable} por Extensómetro"
        x_col, y_col = 'FECHA', variable
        x_title, y_title = "Fecha", variable

    else:
        print("Instrumento no reconocido")
        return

    # Para instrumentos que no sean inclinómetros ni piezómetros
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['FECHA', variable]).copy()
    if df.empty:
        print("⚠️ No hay datos para graficar.")
        return

    fig = go.Figure()
    elementos = sorted(df[col].unique())
    colores = {e: paleta_dropdown.value[i % len(paleta_dropdown.value)] for i, e in enumerate(elementos)}

    modo = {
        "Curvas suaves (spline)": ("lines", "spline"),
        "Líneas rectas": ("lines", "linear"),
        "Puntos": ("markers", None),
        "Líneas + Puntos": ("lines+markers", "linear"),
        "Área apilada": ("lines", "linear"),
        "Área + Líneas": ("lines", "linear"),
        "Área + Líneas + Puntos": ("lines+markers", "linear")
    }
    modo_graf, line_shape = modo[estilo_dropdown.value]
    fill = "tozeroy" if "Área" in estilo_dropdown.value else None
    stackgroup = "one" if estilo_dropdown.value == "Área apilada" else None

    for elem in elementos:
        subdf = df[df[col] == elem].copy()
        fig.add_trace(go.Scatter(
            x=subdf[x_col], y=subdf[y_col],
            mode=modo_graf, name=elem,
            line=dict(width=grosor_dropdown.value, color=colores[elem], shape=line_shape),
            marker=dict(color=colores[elem]),
            fill=fill, stackgroup=stackgroup
        ))

    fig.update_layout(
        width=tamanio_dropdown.value[0], height=tamanio_dropdown.value[1],
        title=titulo, xaxis_title=x_title, yaxis_title=y_title,
        hovermode="x unified", legend_title=col
    )
    fig.show()

# === GUARDAR ===
def guardar_grafica(b=None):
    with output_guardar:
        clear_output()
        ext = formato_dropdown.value
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        if 'fig' not in globals():
            print("⚠️ Primero generá la gráfica.")
            return
        try:
            if ext in [".png", ".jpg", ".svg", ".pdf"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instalá kaleido:\n%pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif ext == ".html":
                fig.write_html(ruta)
            print(f"✅ Guardado: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# === EVENTOS ===
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
boton.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === DISPLAY FINAL ===
display(HTML("<h3 style='color:#1866a3'>Visualización Interactiva</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))

# Fila 1: Selectores principales por instrumento
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))

# Fila 2: Variables y años
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))

# Fila 3: Opciones de gráfica
display(widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]))

display(boton)
display(output)
display(widgets.HBox([formato_dropdown, ruta_text, boton_guardar]))
display(output_guardar)

# Inicializar
actualizar_controles_visibles()

Button(button_style='success', description='Graficar', style=ButtonStyle())

Output()

Output()